In [1]:
import json
import pickle
from collections import defaultdict
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import matplotlib.patches as patches
from tqdm import tqdm
import os

In [ ]:
#load predicted relations(attributes_with) on Test including gt
with open("./Outputs/imat_attr_test_gt_step499999/rel_detections.pkl","rb") as f:
    rel_output = pickle.load(f)
with open('./data/imaterialist/predicates_attributes.json') as f:#list of predicates (edges of the graph) which includes "with" + "all attributes"
    prd_cats = json.load(f)
with open('./data/imaterialist2/test_detections.json', 'rb') as handle:
    categories = json.load(handle)
    categories = categories["categories"]
    categories_id_to_name = {cat["id"]:cat["name"] for cat in categories}

In [ ]:
with open("./data/imaterialist2/mask_id_to_color2.json","r+") as f:
    img_to_color = json.load(f)
color_dct = {1:"black",2:"blue",3:"green",4:"cyan",5:"red",6:"maginta",7:"yellow",8:"white"}

In [ ]:
"""with open('./data/imaterialist2/predicates_counter.json') as f:
    predicate_counter = json.load(f)
sum_c = sum(list(predicate_counter.values())[1:])
attr_waights = np.array([sum_c/predicate_counter[str(i)] if str(i) in predicate_counter and sum_c/predicate_counter[str(i)]<1000 else 0 for i in range(len(prd_cats))])

"""
main_categories = {cat["id"]:cat["name"] for cat in categories if cat["supercategory"] in ['upperbody','lowerbody','wholebody']}
main_categories

In [ ]:
def get_limits_of_bbox(sbj_bboxes,obj_boxes):
    sbj_bboxes = np.asarray(sbj_bboxes)
    obj_boxes = np.asarray(obj_boxes)
    list_bboxes = np.concatenate((sbj_bboxes,obj_boxes))
    Xs, Ys, Ws, Hs =  list_bboxes.T
    crop_right = min (Xs) 
    crop_upper = min(Ys)
    crop_left = max(Xs+Ws)
    crop_lower = max(Ys+Hs)
    return (crop_right-2,crop_upper-2,crop_left+2,crop_lower+2)
get_limits_of_bbox(rel_output[78 ]['gt_sbj_boxes'] , rel_output[78]['gt_obj_boxes'])  

In [ ]:
class SG2Vec:
    """
    #score1= ***_obj_membership = what (main) objects appear (0 or 1) 
    #score2= what hierarchies appear (rel with) (score > thresh_with)
    #score3= what attributes (only main objects?)(score >thresh_attr)
    #score4= color(object,rgb)
    #similarity = (s1_gt dot s1_2 +  s2_gt dot s2_2  +  s3_gt dot s3_2  +  s4_gt dot s4_2)
    """
    
    def __init__(self, objects, main_objects, predicates,thr_obj=0.2, thr_with=0.1, thr_attr=0.09):
        
        self.objects = objects
        self.num_obj = len(self.objects)
        self.main_objects = main_objects
        self.num_main_obj = len(self.main_objects) 
        self.predicates = predicates
        self.num_prd = len(self.predicates)#with + attr
        self.thr_with = thr_with
        self.thr_attr = thr_attr
        self.thr_obj = thr_obj

    def _encode_color(self,rgb):
        """rgb to 8-color pallete"""
        r = "1" if rgb[0] > 127 else "0"
        g = "1" if rgb[1] > 127 else "0"
        b = "1" if rgb[2] > 127 else "0"
        
        for i in range(8):
            if r+g+b == format(i, '03b'):
                return i+1
            
    def get_scores(self, img_relations, color_list ):
        #common between gt and pred
        main_obj_index = { original_id:i for i, original_id in enumerate(self.main_objects.keys())}
        
        #1. gt annotations
        #1.1 gt_score1
        gt_obj_membership = np.zeros(self.num_main_obj)
        #1.2 gt_score2
        gt_rel_membership = np.zeros((self.num_obj , self.num_obj))
        #1.3 gt_score3
        gt_attr_membership = np.zeros((self.num_obj , self.num_prd))#including with
        #1.4 gt_score4
        gt_color_membership = np.zeros(self.num_obj )#3bit color=8 colors
        for maskid,catid,color in color_list:
            gt_color_membership[catid] = self._encode_color(color)
                
        
        for gt_prd, gt_sbj, gt_obj in zip(img_relations['gt_prd_labels'],img_relations['gt_sbj_labels'],img_relations['gt_obj_labels']):
            #1.1
            if gt_sbj in self.main_objects:
                gt_obj_membership[main_obj_index[gt_sbj]] = 1
            if gt_sbj in self.main_objects:
                gt_obj_membership[main_obj_index[gt_sbj]] = 1
            
            #1.2
            if gt_prd == 0: #with
                gt_rel_membership[gt_sbj , gt_obj] = 1
            
            #1.3
            gt_attr_membership[gt_sbj , gt_prd] = 1
            gt_attr_membership[gt_obj , gt_prd] = 1
            gt_attr_membership[gt_sbj , 0] = 0
            gt_attr_membership[gt_obj , 0] = 0
            
            #1.4


            
        #2. predicted annotations   
        #2.1 pred_score1
        pred_obj_membership = np.zeros(self.num_main_obj)
        #2.2 pred_score2
        pred_rel_membership = np.zeros((self.num_obj , self.num_obj))
        #2.3 pred_score3
        pred_attr_membership = np.zeros((self.num_obj , self.num_prd))#self.num_prd is including with
        for sbj, obj, sbj_score, obj_score, prd_scores in zip(img_relations['sbj_labels'],img_relations['obj_labels'],img_relations['sbj_scores'],img_relations['obj_scores'],img_relations['prd_scores_ttl']):
            #2.1
            if sbj in self.main_objects and sbj_score>0.9:
                pred_obj_membership[main_obj_index[sbj]] = 1
            if obj in self.main_objects and obj_score>0.9:
                pred_obj_membership[main_obj_index[obj]] = 1
            #2.2
            with_score = prd_scores[1]#0th:background #1st: with #2nd:344th attributes
            pred_rel_membership[sbj,obj] = 1 if sbj_score * obj_score > self.thr_obj and with_score > self.thr_with else 0
            
            #2.3
            attr_scores =  prd_scores[1:self.num_prd+1]#excluding background
            attr_scores[0]=0 #excluding with
            attr_scores = np.array([ (0 if sbj_score * obj_score *score < self.thr_attr else 1) for score in attr_scores])
            
            #if sbj_score * obj_score > self.thr_obj and with_score > self.thr_with:
            #    attr_scores[0] = 10 if sbj in self.main_objects or obj in self.main_objects else 1#giving more priority to main clothing 
            
            pred_attr_membership[obj, :] = np.maximum(pred_attr_membership[obj, :],attr_scores)
            pred_attr_membership[sbj, :] = np.maximum(pred_attr_membership[sbj, :],attr_scores)
        
        return {"pred_score1":pred_obj_membership, "pred_score2":pred_rel_membership, "pred_score3":pred_attr_membership,
                "gt_score1":gt_obj_membership, "gt_score2":gt_rel_membership, "gt_score3":gt_attr_membership,"gt_score4":gt_color_membership
               }
        

In [ ]:
#7,14,56,38,26,3890
sg2vec = SG2Vec(categories_id_to_name,categories_id_to_name, prd_cats)
scores = sg2vec.get_scores(rel_output[38 ],img_to_color[os.path.basename(rel_output[1 ]["image"])])
print([(categories_id_to_name[i],prd_cats[j]) for i, j in np.argwhere(scores["gt_score3"] > 0)])
print("")
print([(categories_id_to_name[i],prd_cats[j]) for i, j in np.argwhere(scores["pred_score3"] > 0)])
print("")
print([(categories_id_to_name[i],categories_id_to_name[j]) for i, j in np.argwhere(scores["gt_score2"] > 0)])
print("")
print([(categories_id_to_name[i],categories_id_to_name[j]) for i, j in np.argwhere(scores["pred_score2"] > 0)])
print("")
print([categories_id_to_name[i[0]] for i in np.argwhere(scores["gt_score1"] > 0)])
print("")
print([categories_id_to_name[i[0]]  for i in np.argwhere(scores["pred_score1"] > 0)])
print("")
print([(categories_id_to_name[i[0]],color_dct[scores["gt_score4"][i[0]]]) for i in np.argwhere(scores["gt_score4"] > 0)])